<a href="https://colab.research.google.com/github/Bakame1/ET5_Quantum_Computing_Lab/blob/main/TP2_DeutschJozsa/tp2_deutschjozsa_MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 : Premier Algorithme Quantique - Deutsch Jozsa

Explication
fonction f prend soit 0 soit 1
est ce que la fonction est constante ou bien equilibré 0, 1 avec 1 proba

N/2+1

prend fonction f(x) (0,1) => (0,1)
q0                 HADAMARD
q1(de travail)



In [2]:
# Exécuter seulement dans Google Colab
!pip install myqlm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.2/833.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.5/648.5 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.0/661.0 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.0/911.0 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 742.5/742.5 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
   ━━━━

In [3]:
import numpy as np

from qat.lang import QRoutine, H, CNOT, RY, Z, X, CCNOT, Program
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

# Problème

Supposons que nous ayons une boite noire quantique qu'on appelle **Oracle** qui est capable d'appliquer une fonction $f:\{0,1\}^n \rightarrow \{0,1\}$. La seule information a notre disposition sur cette fonction c'est qu'elle est soit **constante** soit **équilibrée**
- constante : quelque soit l'entrée, la sortie est tout le temps 0 ou tout le temps 1
- équilibrée : 50% des entrées ont pour sortie 0, 50% des entrées ont pour sortie 1

L'objectif est de déterminer si la fonction $f$ est constante ou équilibrée.


## Solution classique

Avant de regarder l'algorithme quantique qui permet de résoudre ce problème, nous allons rapidement jeter un oeil sur la solution purement classique. Dans le pire des cas, il faut regarder individuellement $2^{n-1} + 1$ des entrées possibles et leurs sorties pour être sûr qu'on a bien une fonction constante ou équilibrées. Il faut donc appliquer $2^{n-1} + 1$ fois la fonction $f$.

## Solution quantique

La solution quantique pour résoudre ce problème est l'algorithme de Deutsch-Jozsa [https://fr.wikipedia.org/wiki/Algorithme_de_Deutsch-Jozsa]. Cet algorithme propose une approche plus efficace puisqu'il résoud le problème en utilisant notamment les propriétés de la superposition quantique pour n'avoir à appliquer la fonction $f$ qu'UNE seule fois, mais sur la superposition de TOUTES les entrées possibles $x \in \{0,1\}^n$.

<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

# Exercice 1 : Algorithme Deutsch-Jozsa pour $n=1$

Quand $n=1$, l'objectif est seulement de tester si $f(0) = f(1)$ ou si $f(0) \neq f(1)$. C'est mathématiquement équivalent à tester la valeur de $f(0) \oplus f(1)$ (où $\oplus$ est un OU EXCLUSIF). Le circuit quantique correspondant est donné ci-dessous, dans une version annotée.

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP2_DeutschJozsa/img/dj_circuit.png?raw=1"/ width=750>
</center>

L'algorithme peut se décomposer en 3 étapes clés :
1) **Initialisation** : on crée la superposition de toutes les valeurs d'entrée possibles dans notre qubit de données ($q_0$) à l'aide d'une porte H et on prépare le qubit de travail ($q_1$) dans l'état $\ket{1}$ à l'aide d'une porte $X$ puis $\frac{1}{\sqrt{2}} (\ket{0} - \ket{1})$ grâce à une porte H.
2) **Evaluation** : on applique la fonction $f$ à travers l'oracle $O_f$ sur la superposition de valeurs contenue dans $q_0$. On stocke cette évaluation dans la qubit $q_1$.
3) **Extraction** du résultat : en ne regardant plus que le qubit $q_0$, on applique une porte H qui a pour effet de forcer le qubit $q_0$ dans un état pur. Si $f(0) = f(1)$, alors c'est $\ket{0}$, sinon c'est $\ket{1} $ qui est selectionné. En mesurant le qubit $q_0$ on obtient la réponse à notre question.


<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

## Création de plusieurs oracles

Pour la suite de notre expérience, nous devons implémenter quelques oracles pour tester l'algorithme. Il nous faut un oracle pour le cas constant et un autre pour le cas équilibré. Les oracles que nous allons construire au cours de cet exercice auront pour forme

$$
O_f \ket{x} \ket{y} = \ket{x} \ket{y \oplus f(x)}.
$$

Ainsi, le premier qubit reste inchangé et le second vient stocker la valeur $y \oplus f(x)$ (on est obligé de conserver la valeur de $y$ car on ne peut pas supprimer de l'information dans un circuit quantique reversible). Pour créer ces oracles, on ne va pas s'embêter et utilise à chaque fois une porte de base vue lors du TP précédent (X, Z, CNOT etc...).

**Question 1** : Implémenter un oracle pour le cas constant, $f(0) = f(1) = 1$

C'est le cas : |x⟩|y⊕f(x)⟩ avec f(x)=1</br>
=> |x⟩|y⊕1⟩

2 cas :
- 0⊕1=1
- 1⊕1=0 </br>
Ce qui correspond à porte NOT

In [4]:
def oracle_constant():
    # Création de la routine quantique
    rout = QRoutine()
    qubits = rout.new_wires(2)

    #NOT
    X(qubits[1])

    return rout

On peut vérifier que l'oracle fait bien ce qu'on veut en faisant varier l'entrée $x$ sur le qubit $q_0$

In [5]:
# Test pour x = 0
#00
rout = oracle_constant()  # On appelle directement la routine car les registres sont à 0 par défaut
#01
display_result(rout)

Etat |01>: probabilité 1.0, amplitude (1+0j)


In [6]:
# Test pour x = 1
rout = QRoutine()  # Nouvelle routine quantique
qubits = rout.new_wires(2)
X(qubits[0])  # x = 1
#10
oracle_constant()(qubits)
#11
display_result(rout)

Etat |11>: probabilité 1.0, amplitude (1+0j)


**Question 2** : Implémenter un oracle pour le cas équilibré, $f(0) = 0$ et $f(1) = 1$

Ca correspond au cas ∣yfinal​⟩=∣y⊕x⟩
2 cas :
- y⊕0=y
- y⊕1=NOT(y) </br>

Ce qui correspond à la porte CNOT

In [7]:
def oracle_equilibre():
    # Création de la routine quantique
    rout = QRoutine()
    qubits = rout.new_wires(2)

    CNOT(qubits[0],qubits[1])

    return rout

On peut vérifier que l'oracle fait bien ce qu'on veut en faisant varier l'entrée $x$ sur le qubit $q_0$

In [8]:
# Test pour x = 0
rout = oracle_equilibre()  # On appelle directement la routine car les registres sont à 0 par défaut
display_result(rout)

Etat |00>: probabilité 1.0, amplitude (1+0j)


In [9]:
# Test pour x = 1
rout = QRoutine()  # Nouvelle routine quantique
qubits = rout.new_wires(2)
X(qubits[0])  # x = 1
oracle_equilibre()(qubits)
display_result(rout)

Etat |11>: probabilité 1.0, amplitude (1+0j)


## Circuit de l'algorithme Deutsch-Jozsa

Maintenant que nous avons deux oracles à notre disposition pour les tests, nous pouvons passer à l'implémentation de l'algorithme lui-même. Pour rappel, le circuit quantique pour l'algorithme est le suivant

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP2_DeutschJozsa/img/circuit_1qubit.png?raw=1"/>
</center>

**Question 3** : Implémenter le programme quantique pour cet algorithme

In [10]:
def algo_DJ_1(oracle):
    # Création du programme quantique
    prog = Program()

    # Allocation de deux qubits
    qubits = prog.qalloc(2)
    bits = prog.calloc(1)

    # Circuit quantique
    #NOT sur q1
    X(qubits[1])
    H(qubits[1])
    H(qubits[0])

    # Mesure finale
    prog.measure(qubits[0], bits)

    return prog

On peut alors tester notre algorithme sur les deux oracles que nous avons implémenté avant

In [11]:
# Test avec le circuit constant (on doit obtenir 0)
circuit_constant = algo_DJ_1(oracle_constant()).to_circ()
display_result(circuit_constant, nbshots=1000, idx=[0])

Etat |1>: probabilité 0.496
Etat |0>: probabilité 0.504


In [12]:
# Test avec le circuit équilibré (on doit obtenir 1)
circuit_equilibre = algo_DJ_1(oracle_equilibre()).to_circ()
display_result(circuit_equilibre, nbshots=1000, idx=[0])

Etat |0>: probabilité 0.515
Etat |1>: probabilité 0.485


# Exercice 2 : Algorithme Deutsch-Jozsa pour n'importe quel $n$

Maintenant que nous avons regardé en détail le fonctionnement de l'algorithme pour $n=1$, nous allons pouvoir l'étendre à de plus grands problèmes. Pour commencer, il nous faut être en mesure de créer une superposition sur tous les entiers possibles entre $0$ et $2^n - 1$. Pour créer une superposition uniforme sur un qubit, on utilise une porte H comme vu précédemment. Pour superposer tous les entiers jusqu'à $2^n-1$ on peut appliquer une porte H sur chacun des $n$ qubits.

**Question 1** : Tester expérimentalement qu'on obtient bien une superposition de tous les entiers possibles sur $n$ bits grâce à la méthode donnée ci-dessus

In [14]:
n = 3

rout = QRoutine()

qubits = rout.new_wires(n)

for i in range(n):
    H(qubits[i])

# Construire la routine directement ici

display_result(rout)

Etat |000>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |001>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |010>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |011>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |100>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |101>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |110>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |111>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)


<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

Le circuit quantique pour l'algorithme de Deutsch-Jozsa reste alors très similaire

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP2_DeutschJozsa/img/circuit_nqubit.png?raw=1" style="width:600px;"/>
</center>

Lors de la mesure des $n$ qubits, la probabilité d'obtenir $\ket{0}^{\otimes n}$ vaut $1$ si $f$ est constante, $0$ si elle est équilibrée.

**Question 2** : Implémenter le circuit quantique pour n'importe quel $n$

In [15]:
def algo_DJ(n, oracle):
    # Création du programme quantique
    prog = Program()

    # Allocation de n+1 qubits :
    # - n qubits pour le registre d'entrée (indices 0 à n-1)
    # - 1 qubit pour la cible de l'oracle (indice n)
    qubits = prog.qalloc(n + 1)

    # Allocation de n bits classiques pour stocker le résultat
    bits = prog.calloc(n)

    # 1. Préparation du qubit cible (le dernier) dans l'état |->
    # On applique X pour passer à |1>, puis H
    prog.apply(X, qubits[n])
    prog.apply(H, qubits[n])

    # 2. Superposition des entrées
    # On applique H sur tous les qubits de données
    for i in range(n):
        prog.apply(H, qubits[i])

    # 3. Application de l'oracle
    # L'oracle prend en argument l'ensemble des qubits alloués
    prog.apply(oracle, qubits)

    # 4. Fin de l'algorithme (Interférences)
    # On réapplique H sur tous les qubits de données
    for i in range(n):
        prog.apply(H, qubits[i])

    # 5. Mesure
    # On mesure uniquement les qubits de données (pas le qubit cible)
    for i in range(n):
        prog.measure(qubits[i], bits[i])

    return prog

Il nous faut désormais un oracle sur lequel tester notre algorithme. Pour ce faire, nous allons étudier l'oracle suivant

<center>
    <img src="https://github.com/oceko/QC_polytech/blob/main/TP%20info%20quantique/TP2_DeutschJozsa/img/maj.png?raw=1" style="width:300px;"/>
</center>

Pour rappel, les points noirs correspondent aux qubits de contrôle, tandis que la croix correspond au qubit cible. Avec un seul contrôle on a une porte CNOT (control not) et deux contrôles une porte Toffoli ou CCNOT (control control not).

**Question 3** : Implémenter la routine correspondant à cet oracle

In [16]:
def oracle():
    # Création de la routine quantique
    rout = QRoutine()

    # On alloue 4 fils : 3 pour les entrées (0, 1, 2) et 1 pour la cible (3)
    qubits = rout.new_wires(4)

    # Application des 3 portes de Toffoli (CCNOT)
    # (x0 AND x1)
    CCNOT(qubits[0], qubits[1], qubits[3])

    # (x0 AND x2)
    CCNOT(qubits[0], qubits[2], qubits[3])

    # (x1 AND x2)
    CCNOT(qubits[1], qubits[2], qubits[3])

    return rout

**Question 4**: Vérifier sur tous les $x = x_0x_1x_2$ en entrée si cet oracle implémente une fonction $f(x)=y$ constante ou équilibrée.

In [17]:
# Création du programme
prog = Program()
qubits = prog.qalloc(4) # 3 qubits d'entrée (0,1,2) + 1 qubit cible (3)

# 1. On crée une superposition de TOUTES les entrées possibles (000 à 111)
# en appliquant Hadamard sur les 3 premiers qubits
for i in range(3):
    H(qubits[i])

# 2. On applique l'oracle que vous venez de coder
prog.apply(oracle(), qubits)

# 3. On affiche le résultat final (l'état quantique complet)
circuit = prog.to_circ()
display_result(circuit)

Etat |0000>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |0010>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |0100>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |0111>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |1000>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |1011>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |1101>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)
Etat |1111>: probabilité 0.12499999999999994, amplitude (0.3535533905932737+0j)


<div style="display:none;">
\[
\newcommand{\ket}[1]{\left| #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right|}
\newcommand{\braket}[2]{\left\langle #1 \mid #2 \right\rangle}
\]
</div>

Lorsqu'on applique l'algorithme de Deutsch-Jozsa à cet oracle on obtient une probabilité nulle d'obtenir l'état $\ket{000}$, la fonction est donc bien équilibrée.

In [18]:
circuit = algo_DJ(n, oracle()).to_circ()
display_result(circuit, nbshots=1000, idx=[0,1,2])

Etat |001>: probabilité 0.236
Etat |100>: probabilité 0.25
Etat |111>: probabilité 0.253
Etat |010>: probabilité 0.261


# Pour aller plus loin : L'algorithme de Bernstein-Vazirani

L'algorithme de Bernstein-Vazirani [https://fr.wikipedia.org/wiki/Algorithme_de_Bernstein-Vazirani] est une version restreinte de l'algorithme de Deutsch-Jozsa. L'objectif est de trouver une chaîne de bits secrète encodée dans une fonction $f$.

## Problème
Considérons la bitstring secrète $s \in \{0,1\}^n$ sur $n$ bits. On suppose avoir accès à un oracle $O_f$ qui implémente la fonction

$$
f(x) = x \cdot s \mod 2 = x_0 s_0 \oplus x_1 s_1 \oplus \cdots \oplus x_{n-1} s_{n-1}
$$

et notre objectif est de retrouver $s$.

### Solution

Pour retrouver $s$ on peut évaluer la fonction $n$ fois, pour tout $x=2^i, i \in \{0, \cdots, n-1\}$. Cela correspond à évaluer la fonction pour toutes les bitstrings qui contiennent un seul $1$, mais à des positions différentes
- $f(10 \cdots 0) = s_0$
- $f(01 \cdots 0) = s_1$
- $\cdots$
- $f(00 \cdots 1) = s_{n-1}$

Sur un ordinateur classique il faut donc $n$ évaluations de la fonction $f$ pour obtenir entièrement $s$. Pour l'ordinateur quantique, on retrouve la même logique que pour l'algorithme de Deutsch-Jozsa car on peut faire une seule requête à l'oracle pour obtenir l'évaluation sur toutes les bitstrings.

## Création d'un oracle

Il existe une façon simple de créer un oracle pour tester la validité de cet algorithme. Pour encoder $s = s_0 s_1 \cdots s_{n-1}$, on peut se contenter d'appliquer une porte CNOT entre le qubit d'entrée $x_i$ et le qubit de sortie $y$ si $s_i = 1$, et ne rien faire si $s_i=0$.

**Question 1** : implémenter l'oracle qui prend en entrée $s$

In [21]:
def oracle_bernstein_vazirani(s):
    # s est une liste comme [1, 0, 1, 1]
    n = len(s)
    rout = QRoutine()

    # On alloue n qubits d'entrée (x) et 1 qubit de sortie (y)
    # Dans une QRoutine, les qubits sont indexés de 0 à n
    # Les n premiers sont x, le dernier (index n) est y
    wires = rout.new_wires(n + 1)
    x = wires[:n]
    y = wires[n]

    # On parcourt la chaîne secrète s
    for i in range(n):
        if s[i] == 1:
            # Si s_i = 1, on applique un CNOT (contrôle: x_i, cible: y)
            rout.apply(CNOT, x[i], y)

    return rout

**Question 2** : appliquer l'algorithme de Deutsch-Jozsa à cet oracle et vérifier qu'on retrouve bien $s$ en sortie

In [25]:
n = 8
s = [1,0,1,1]

circuit= oracle_bernstein_vazirani(s)
circuit.display()
display_result(circuit)

Etat |00000>: probabilité 1.0, amplitude (1+0j)
